In [1]:
import pandas as pd
import numpy as np
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-11 23:53:22,0.43945,0.0,0.0,0.0,2022-06-11,23:53:22,23
1,2022-06-11 23:54:22,0.48828,0.0,0.0,0.0,2022-06-11,23:54:22,23
2,2022-06-11 23:55:22,0.46387,0.0,0.0,0.0,2022-06-11,23:55:22,23
3,2022-06-11 23:56:22,0.46387,0.0,0.0,0.0,2022-06-11,23:56:22,23
4,2022-06-11 23:57:22,0.46387,0.0,0.0,0.0,2022-06-11,23:57:22,23


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
13163,2022-06-21 11:31:29,2022-06-21,11:31:29,11,20.67871,0.00000,0.000000,0.000000
13164,2022-06-21 11:32:29,2022-06-21,11:32:29,11,20.67871,0.00000,0.000000,0.000000
13165,2022-06-21 11:33:29,2022-06-21,11:33:29,11,16.01563,5.04290,80.765221,0.080765
13166,2022-06-21 11:34:29,2022-06-21,11:34:29,11,14.47754,1.73918,25.179048,0.025179
13167,2022-06-21 11:35:30,2022-06-21,11:35:30,11,14.30664,1.59849,22.869021,0.022869


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-11T00:00:00.000000000', '2022-06-12T00:00:00.000000000',
       '2022-06-13T00:00:00.000000000', '2022-06-14T00:00:00.000000000',
       '2022-06-15T00:00:00.000000000', '2022-06-16T00:00:00.000000000',
       '2022-06-17T00:00:00.000000000', '2022-06-18T00:00:00.000000000',
       '2022-06-19T00:00:00.000000000', '2022-06-20T00:00:00.000000000',
       '2022-06-21T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

In [7]:
filter_daily_values = df[(df['Date'] >= '2022-06-18') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
8169,2022-06-18,0,0.0
8170,2022-06-18,0,0.0
8171,2022-06-18,0,0.0
8172,2022-06-18,0,0.0
8173,2022-06-18,0,0.0
...,...,...,...
12469,2022-06-20,23,0.0
12470,2022-06-20,23,0.0
12471,2022-06-20,23,0.0
12472,2022-06-20,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values.head()

,Date,Hour,Power (KW)
0,2022-06-18,0,0.0
1,2022-06-18,1,0.0
2,2022-06-18,2,0.0
3,2022-06-18,3,0.0
4,2022-06-18,4,0.0


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data.drop(['RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
weather_data.tail()

,SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
91,5.8,22.8,35,46
92,4.9,21.6,39,11
93,1.5,19.9,48,11
94,0.0,18.3,57,11
95,0.0,16.6,66,11


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail()

,Power (KW),SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
91,NaN,5.8,22.8,35,46
92,NaN,4.9,21.6,39,11
93,NaN,1.5,19.9,48,11
94,NaN,0.0,0.0,0,0
95,NaN,0.0,0.0,0,0


In [12]:
count_total_rows = len(df1) - 24

In [13]:
independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]

In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
reg = linear_model.LinearRegression()
reg.fit(independent_columns, dependent_column)

LinearRegression()

In [16]:
forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(24)

In [17]:
return_array = list(reg.predict(forcasted_data))

In [18]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [19]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [20]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [21]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [22]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-06-21,0,0.013957
1,2022-06-21,1,0.013957
2,2022-06-21,2,0.013957
3,2022-06-21,3,0.013957
4,2022-06-21,4,0.013957
5,2022-06-21,5,-0.124996
6,2022-06-21,6,-0.002000
7,2022-06-21,7,0.142894
8,2022-06-21,8,-0.085434
9,2022-06-21,9,0.370241


In [23]:
# dependent_variable = 'Power (KW)'

In [24]:
# independent_variables = df1.columns.tolist()
# independent_variables.remove(dependent_variable)

In [25]:
# independent_variables

In [26]:
# X = df1[independent_variables].values

In [27]:
# y = df1[dependent_variable].values

In [28]:
# X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [29]:
# X_train

In [30]:
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

In [31]:
# y_pred = regressor.predict(X_test)
# y_pred

In [32]:
# math.sqrt(mean_squared_error(y_test, y_pred))

In [33]:
# r2_score(y_test, y_pred)